In [ ]:
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tiled.client import from_uri
from tiled.queries import Key

from tqdm.notebook import tqdm
import aimmdb

In [ ]:
# the tiled client object represents the data on the server in the form of a tree which can be traversed using python's dictionary syntax
# at the root level the keys correspond to available queries
c = from_uri("https://aimm.lbl.gov/api")
c

In [ ]:
# data on the server is organized into datasets
c["dataset"]

In [ ]:
# login to see non public datasets
c.login()

In [ ]:
c["dataset"]

In [ ]:
# choosing a query shows the distinct values that the query parameter can take
# uid gives a flat list of all selected data
c["uid"]

In [ ]:
# the keys can be indexed using the .keys() method
c["uid"].keys()[:15]

In [ ]:
# measurements are indexed by a unique identifier (uid)
x = c["uid"]["95NqB8p5xEd"]
x

In [ ]:
# or by using the .values() method
x = c["uid"].values().first()
x

In [ ]:
# measurements have an associated metadata dictionary
x.metadata

In [ ]:
# the measurement itself is stored in a dataframe which can be obtained using the .read() method
df = x.read()
df

In [ ]:
# the dataset query gives a list of datasets (groupings of data within the database)
c["dataset"]

In [ ]:
# selecting a dataset yields a client object like the root client but with the dataset query removed
c["dataset"]["newville"]

In [ ]:
# the uid query after selecting a dataset gives only the measurements which are part of that dataset
c["dataset"]["newville"]["uid"]

In [ ]:
# the element query gives a list of distinct absorbing elements
c["element"]

In [ ]:
# queries may be nested
# for example querying the list of distinct absorbing elements present in the newville dataset can be written as
c["dataset"]["newville"]["element"]

In [ ]:
# and selecting the gold spectra within the newville dataset can be written as
c["dataset"]["newville"]["element"]["Au"]["uid"].keys()[:]

In [ ]:
# measurements can also be grouped by sample
# samples are also identified by a uid
c["dataset"]["newville"]["sample"].keys()[:10]

In [ ]:
# samples may be selected by their uid and can have more than one measurement associated with them
c["dataset"]["newville"]["sample"]["745QhDdAY9Y"]["uid"].keys()[:]

In [ ]:
# the database can also be search by edge ...
c["dataset"]["aimm_core"]["edge"]

In [ ]:
# ... and this can be nested
c["dataset"]["aimm_core"]["element"]["O"]["edge"]["K"]["uid"].keys()[:]

In [ ]:
# once the uid has been found, only the uid is necessary to select the desired measurement
x = c["uid"]["5j5ZLA6Qokx"]
x

In [ ]:
# pandas provides convenient functions for plotting dataframes

pprint({**x.metadata})
df = x.read()

fig, ax = plt.subplots()
df.plot("energy", "mu", ax=ax, title=x.metadata["sample"]["name"])

In [ ]:
# the order of traversal of queries is not important
# the following two expressions are fully equivalent

In [ ]:
c["dataset"]["aimm"]["element"]["Ni"]["edge"]["K"]["uid"]

In [ ]:
c["dataset"]["aimm"]["edge"]["K"]["element"]["Ni"]["uid"]

In [ ]:
# the tree mechanism is not the only way to search the database
# the above queries can also be written as

c["uid"].search(Key("dataset") == "aimm").search(Key("element.edge") == "K").search(Key("element.symbol") == "Ni")

In [ ]:
# these search mechanisms are fully composable so the following is also equivalent to the preceeding three examples

c["dataset"]["aimm"]["uid"].search(Key("element.edge") == "K").search(Key("element.symbol") == "Ni")

In [ ]:
# let's look at one of the spectra in the aimm dataset
x = c["dataset"]["aimm"]["uid"].values().first()
x

In [ ]:
# the metadata for this spectra contain information on the charge state
x.metadata

In [ ]:
# the .search(...) method may be used to query metadata which is not searchable using the tree traversal methods
# for example the following query fetchs all data in the aimm dataset corresponding to the pristine condition (i.e. not charge cycled)

c["dataset"]["aimm"]["uid"].search(Key("charge.cycle") == 0)

In [ ]:
# print aimm dataset samples
for k in c["dataset"]["aimm"]["sample"].keys():
    print(k)

In [ ]:
# choose one sample

sample = c["dataset"]["aimm"]["sample"].keys().first()
sample

In [ ]:
# get all measurements one sample (recall samples are identified by their uid not their name)

c["dataset"]["aimm"]["sample"][sample]["uid"]

In [ ]:
# plot all K edge measurements of pristine condition

fig, ax = plt.subplots()

ax.set_title(sample.name)

for v in c["dataset"]["aimm"]["sample"][sample]["uid"].search(Key("charge.cycle") == 0).values():
    df = v.read()
    element = v.metadata["element"]["symbol"]
    edge = v.metadata["element"]["edge"]
    if "mutrans" in df.columns:
        df.plot("energy", "mutrans", ax=ax, label=f"{element}-{edge}")

In [ ]:
# plot all Nickle L3 measurements

fig, ax = plt.subplots()
for v in c["dataset"]["aimm"]["sample"][sample]["element"]["Ni"]["edge"]["L3"]["uid"].values():
    charge = v.metadata["charge"]
    cycle, voltage, state = [charge[k] for k in ["cycle", "voltage", "state"]]
    df = v.read()
    df.plot("energy", "mu_tey", ax=ax, label=f"cycle {cycle} {voltage}V {state}")

In [ ]:
# plot Manganese K edge for pristine condition for each sample

fig, ax = plt.subplots()

for v in c["dataset"]["aimm"]["element"]["Mn"]["edge"]["K"]["uid"].search(Key("charge.cycle") ==  0).values():
    df = v.read()
    sample_name = v.metadata["sample"]["name"]
    
    df.plot("energy", "mutrans", ax=ax, label=f"{sample_name}")    

In [ ]:
# find Ni-L3 edge in aimm_core dataset
c["dataset"]["aimm_core"]["element"]["Ni"]["edge"]["L3"]["uid"]

In [ ]:
# compare Ni-L3 edge in Ni metal with BM_NCM622

In [ ]:
x = c["dataset"]["aimm_core"]["element"]["Ni"]["edge"]["L3"]["uid"].values().first()
x

In [ ]:
y = c["dataset"]["aimm"]["sample"][sample]["element"]["Ni"]["edge"]["L3"]["uid"].search(Key("charge.cycle") == 0).values().first()
y

In [ ]:
fig, ax = plt.subplots()

df = x.read()
df.plot("energy", "mu_tey", ax=ax, label=x.metadata["sample"]["name"])

df = y.read()
df.plot("energy", "mu_tey", ax=ax, label=y.metadata["sample"]["name"])